In [19]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
import pathlib
from pathlib import Path
import matplotlib.pyplot as plot
import librosa
    
# import the necessary packages
from musicrec.vgg import VGGNet
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
from datetime import datetime
import pickle
import cv2
import os
import sys

from keras.models import Sequential

In [23]:

data_folder = Path("../../../audio/testfiles/GTZAN_test/genres/")
output_folder = Path("./output/cvnn.model")
spectogram_folder = Path("./img_data/")
# Duration of songsnippet in seconds
duration = 10
# Matplotlib colormap for spectogram
spectogram_cmap = 'binary' 
# Predefined list of genres
pred_genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split() 
#imagePaths = sorted(list(spectogram_folder.list_images(args["dataset"])))
#imagePaths

imagesize_x = 256
imagesize_y = 256

In [24]:
#Get directories of all songs
songs = []
genres = []

spectograms = []

for g in data_folder.iterdir():
    genres.append(g.name)
    for i in g.iterdir():
        songs.append(i)

In [26]:
# Calculate all spectograms
cmap = plot.get_cmap(spectogram_cmap)
plot.figure(figsize=(2.5,5))
spectograms = []


# Iterate through all songs and generate their spactograms. Save them all as images.
for genre in genres:
    pathlib.Path(f'img_data/{genre}').mkdir(parents=True, exist_ok=True)     
for song in songs:
    y, sr = librosa.load(song, mono=True, duration=duration)
    plot.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
    plot.axis('off');
    spec_path = pathlib.Path(f'img_data/{song.parts[-2]}/{".".join(str(song.parts[-1]).split(".")[:2])}.png')
    spectograms.append(spec_path)
    plot.savefig(spec_path, bbox_inches='tight', pad_inches = 0)
    plot.clf()

In [27]:
# Seed for RNG
random.seed(datetime.now())

In [28]:
# Shuffle the spectograms
random.shuffle(spectograms)
spectograms

[PosixPath('img_data/reggae/reggae.00003.png'),
 PosixPath('img_data/reggae/reggae.00004.png'),
 PosixPath('img_data/hiphop/hiphop.00005.png'),
 PosixPath('img_data/blues/blues.00003.png'),
 PosixPath('img_data/hiphop/hiphop.00002.png'),
 PosixPath('img_data/rock/rock.00003.png'),
 PosixPath('img_data/hiphop/hiphop.00001.png'),
 PosixPath('img_data/reggae/reggae.00000.png'),
 PosixPath('img_data/rock/rock.00004.png'),
 PosixPath('img_data/country/country.00000.png'),
 PosixPath('img_data/blues/blues.00001.png'),
 PosixPath('img_data/metal/metal.00001.png'),
 PosixPath('img_data/hiphop/hiphop.00003.png'),
 PosixPath('img_data/pop/pop.00002.png'),
 PosixPath('img_data/jazz/jazz.00000.png'),
 PosixPath('img_data/jazz/jazz.00002.png'),
 PosixPath('img_data/blues/blues.00002.png'),
 PosixPath('img_data/metal/metal.00003.png'),
 PosixPath('img_data/pop/pop.00005.png'),
 PosixPath('img_data/country/country.00005.png'),
 PosixPath('img_data/disco/disco.00003.png'),
 PosixPath('img_data/classic

In [29]:
# Import images, convert them to grayscale (one uint8 per pixel) and load them into an array
images = []
for spec_path in spectograms:
    image = cv2.imread(str(spec_path))
    col_pixels = np.array(np.where(image != 255))
    first_col_pixel = col_pixels[:,0]
    last_col_pixel = col_pixels[:,-1]
    image = image[first_col_pixel[0]:last_col_pixel[0], first_col_pixel[1]:last_col_pixel[1]]
    #image = cv2.resize(image, (imagesize_x, imagesize_y))
    imagesize_x = image.shape[1]
    imagesize_y = image.shape[0]
    image_name = str(spec_path) + "-crop.png"
    cv2.imwrite(image_name, image)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    images.append(image)
images = np.array(images, dtype="float") / 255.0 
data = images

In [30]:
# Create array with correct labels for each spectogram
labels = []
for spec in spectograms:
    labels.append(spec.parts[-2])  
labels

['reggae',
 'reggae',
 'hiphop',
 'blues',
 'hiphop',
 'rock',
 'hiphop',
 'reggae',
 'rock',
 'country',
 'blues',
 'metal',
 'hiphop',
 'pop',
 'jazz',
 'jazz',
 'blues',
 'metal',
 'pop',
 'country',
 'disco',
 'classical',
 'reggae',
 'jazz',
 'country',
 'rock',
 'jazz',
 'hiphop',
 'classical',
 'rock',
 'classical',
 'jazz',
 'blues',
 'disco',
 'metal',
 'rock',
 'rock',
 'disco',
 'metal',
 'pop',
 'pop',
 'blues',
 'disco',
 'country',
 'classical',
 'country',
 'disco',
 'pop',
 'blues',
 'metal',
 'hiphop',
 'reggae',
 'classical',
 'metal',
 'country',
 'classical',
 'reggae',
 'jazz',
 'pop',
 'disco']

In [31]:
np.shape(data)

(60, 384, 193, 3)

In [38]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)
 
# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)
np.size(trainY)
trainY

array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0,

In [33]:
lb.classes_

array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
       'metal', 'pop', 'reggae', 'rock'], dtype='<U9')

In [34]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, 
                         height_shift_range=0.1, shear_range=0.2, 
                         zoom_range=0.2,horizontal_flip=True, 
                         fill_mode="nearest")
 
# initialize our VGG-like Convolutional Neural Network
model = VGGNet.build(width=imagesize_x, height=imagesize_y, depth=3, 
                          num_genres=len(lb.classes_))

In [35]:
# initialize our initial learning rate, # of epochs to train for,
# and batch size
INIT_LR = 0.01
EPOCHS = 550
BS = 32
 
# initialize the model and optimizer (you'll want to use
# binary_crossentropy for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
 
# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS), 
                        validation_data=(testX, testY), 
                        steps_per_epoch=len(trainX) // BS,
                        epochs=EPOCHS)

[INFO] training network...
Epoch 1/550
1/1 [==============================] - 9s 9s/step - loss: 4.4095 - acc: 0.1250 - val_loss: 4.3980 - val_acc: 0.2000
Epoch 2/550
1/1 [==============================] - 3s 3s/step - loss: 4.4085 - acc: 0.0000e+00 - val_loss: 4.4209 - val_acc: 0.1333
Epoch 3/550
1/1 [==============================] - 7s 7s/step - loss: 4.4101 - acc: 0.1250 - val_loss: 4.4048 - val_acc: 0.1333
Epoch 4/550
1/1 [==============================] - 3s 3s/step - loss: 4.4391 - acc: 0.0000e+00 - val_loss: 4.4074 - val_acc: 0.0667
Epoch 5/550
1/1 [==============================] - 7s 7s/step - loss: 4.4269 - acc: 0.0000e+00 - val_loss: 4.4021 - val_acc: 0.0667
Epoch 6/550
1/1 [==============================] - 3s 3s/step - loss: 4.4084 - acc: 0.0000e+00 - val_loss: 4.3957 - val_acc: 0.0667
Epoch 7/550
1/1 [==============================] - 7s 7s/step - loss: 4.4051 - acc: 0.0000e+00 - val_loss: 4.3954 - val_acc: 0.1333
Epoch 8/550
1/1 [==============================] - 3s 3s/

Epoch 64/550
1/1 [==============================] - 4s 4s/step - loss: 4.2617 - acc: 0.2308 - val_loss: 4.3289 - val_acc: 0.0667
Epoch 65/550
1/1 [==============================] - 9s 9s/step - loss: 4.2455 - acc: 0.1875 - val_loss: 4.3170 - val_acc: 0.0667
Epoch 66/550
1/1 [==============================] - 4s 4s/step - loss: 4.3186 - acc: 0.0000e+00 - val_loss: 4.3082 - val_acc: 0.1333
Epoch 67/550
1/1 [==============================] - 7s 7s/step - loss: 4.2800 - acc: 0.1562 - val_loss: 4.3177 - val_acc: 0.1333
Epoch 68/550
1/1 [==============================] - 3s 3s/step - loss: 4.2251 - acc: 0.3077 - val_loss: 4.3107 - val_acc: 0.0667
Epoch 69/550
1/1 [==============================] - 9s 9s/step - loss: 4.2779 - acc: 0.1562 - val_loss: 4.3082 - val_acc: 0.1333
Epoch 70/550
1/1 [==============================] - 4s 4s/step - loss: 4.3449 - acc: 0.1538 - val_loss: 4.2976 - val_acc: 0.0667
Epoch 71/550
1/1 [==============================] - 9s 9s/step - loss: 4.2773 - acc: 0.1562 -

Epoch 128/550
1/1 [==============================] - 4s 4s/step - loss: 4.1910 - acc: 0.1538 - val_loss: 4.1822 - val_acc: 0.1333
Epoch 129/550
1/1 [==============================] - 8s 8s/step - loss: 4.1067 - acc: 0.1562 - val_loss: 4.1859 - val_acc: 0.2000
Epoch 130/550
1/1 [==============================] - 5s 5s/step - loss: 4.1153 - acc: 0.3846 - val_loss: 4.1916 - val_acc: 0.1333
Epoch 131/550
1/1 [==============================] - 9s 9s/step - loss: 4.0424 - acc: 0.2500 - val_loss: 4.2028 - val_acc: 0.1333
Epoch 132/550
1/1 [==============================] - 4s 4s/step - loss: 4.2236 - acc: 0.1538 - val_loss: 4.1720 - val_acc: 0.1333
Epoch 133/550
1/1 [==============================] - 9s 9s/step - loss: 4.1136 - acc: 0.1875 - val_loss: 4.1596 - val_acc: 0.2000
Epoch 134/550
1/1 [==============================] - 4s 4s/step - loss: 4.0358 - acc: 0.1538 - val_loss: 4.1852 - val_acc: 0.1333
Epoch 135/550
1/1 [==============================] - 9s 9s/step - loss: 4.0637 - acc: 0.25

Epoch 191/550
1/1 [==============================] - 8s 8s/step - loss: 3.8696 - acc: 0.3438 - val_loss: 4.0823 - val_acc: 0.1333
Epoch 192/550
1/1 [==============================] - 3s 3s/step - loss: 4.0193 - acc: 0.2308 - val_loss: 4.0736 - val_acc: 0.1333
Epoch 193/550


KeyboardInterrupt: 

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), 
                            target_names=lb.classes_))
 
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
#plt.savefig(args["plot"])
 
# save the model and label binarizer to disk
#print("[INFO] serializing network and label binarizer...")
#model.save(args["model"])
#f = open(args["label_bin"], "wb")
#f.write(pickle.dumps(lb))
#f.close()